In [ ]:
!pip install tf-explain

In [ ]:
import tf_explain
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.pyplot as plt
import os
import random
from sklearn import model_selection as sk_model_selection

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import activations
from tensorflow.keras import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from keras_preprocessing.image.dataframe_iterator import DataFrameIterator
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score


In [ ]:
class Config:
    INPUT_PATH_DCM = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/'
    INPUT_PATH_PNG = '../input/rsna-miccai-png/'
    TENSORBOARD_LOG_DIR = '../working/log_tensorboard/'
    SEED = 42
    #These should be removed from the dataset
    EXCLUDED_STR = ['00109', '00123', '00709']
    EXCLUDED_INT = [109, 123, 709]

    #Defining target size of image
    IMG_SIZE = 224
    NUM_SLICES_3D = 64
    MIN_SLICES = 12
    
    BATCH_SIZE = 64
    
    CLASS_MODE = 'binary'
    COLOR_MODE = 'rgb'
    TARGET_SIZE = (224, 224)
    def __self__():
        pass
    @staticmethod
    def set_seed(seed_val):
        tf.random.set_seed(seed_val)
        random.seed(seed_val)
        os.environ['PYTHONHASHSEED'] = str(seed_val)
        np.random.seed(seed_val)
        

In [ ]:
Config.set_seed(Config.SEED)


# Getting to know the data

In [ ]:
!ls ../input/rsna-miccai-brain-tumor-radiogenomic-classification/


In [ ]:
df = pd.read_csv(Config.INPUT_PATH_DCM+'train_labels.csv', dtype={
    'BraTS21ID': str, 'MGMT_value':int
})

df_test = pd.read_csv(Config.INPUT_PATH_DCM+'sample_submission.csv', dtype={
    'BraTS21ID': str, 'MGMT_value':float
})


df.head()

In [ ]:
#Lendo dataset 
df = df[~df['BraTS21ID'].isin(Config.EXCLUDED_STR)]

#Para testes, usando 1/3 do dataset
# df = df[:int(len(df)/3)]


print(df.shape)

In [ ]:
# adapted from: https://www.kaggle.com/leandronidas/tf-efficientnet-transfer-learning-strat-split/edit

df['flair'] = df['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'train/'+file_id+'/FLAIR/')
df['t1w'] = df['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'train/'+file_id+'/T1w/')
df['t1wce'] = df['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'train/'+file_id+'/T1wCE/')
df['t2w'] = df['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'train/'+file_id+'/T2w/')

df['brats21idInt'] = df['BraTS21ID'].astype(int)

df_test['flair'] = df_test['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'test/'+file_id+'/FLAIR/')
df_test['t1w'] = df_test['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'test/'+file_id+'/T1w/')
df_test['t1wce'] = df_test['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'test/'+file_id+'/T1wCE/')
df_test['t2w'] = df_test['BraTS21ID'].apply(lambda file_id : Config.INPUT_PATH_PNG+'test/'+file_id+'/T2w/')

df_test['brats21idInt'] = df_test['BraTS21ID'].astype(int)

In [ ]:
sample_y = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv')
pred = sample_y
pred['BraTS21ID5'] = [format(x, '05d') for x in pred.BraTS21ID]
pred.head(5)


In [ ]:
#Divisao estratificada em treino, teste e validação
df_trainval, df_test = sk_model_selection.train_test_split(
    df, 
    test_size=0.15, 
    random_state=Config.SEED, 
    stratify=df["MGMT_value"],
)
df_train, df_val = sk_model_selection.train_test_split(
    df_trainval, 
    test_size=0.2, 
    random_state=Config.SEED, 
    stratify=df_trainval["MGMT_value"],
)

print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

In [ ]:
df_val

In [ ]:
print(df_val[df_val['brats21idInt'] == 109].any())

print(df_train[df_train['brats21idInt'] == 109].any())

In [ ]:
    
def get_iterating_dataframe(df, mri_type):
    
    all_test_img_files = []
    all_test_img_labels = []
    all_test_img_patient_ids = []
    for row in df.iterrows():

        img_dir = row[1][mri_type]
        img_files = os.listdir(img_dir)
        img_nums = sorted([int(ele.replace('Image-', '').replace('.png', '')) for ele in img_files])
        totalnum_images = len(img_nums)
        mid_point = int(totalnum_images//2)
        start_point = mid_point - max(int(mid_point*0.1), Config.MIN_SLICES//2)
        end_point = mid_point + max(int(mid_point*0.1), Config.MIN_SLICES//2)
#         print(type(img_nums[0]))
#         break
        img_names = [f'Image-{img_nums[i]}.png' for i in range(start_point, end_point+1)]

        img_paths = [img_dir+ele for ele in img_names]
        img_labels = [row[1]['MGMT_value']]*len(img_paths)
        img_patient_ids = [row[1]['brats21idInt']]*len(img_paths)
        all_test_img_files.extend(img_paths)
        all_test_img_labels.extend(img_labels)
        all_test_img_patient_ids.extend(img_patient_ids)

    new_df = pd.DataFrame({'patient_ids': all_test_img_patient_ids,
                  'labels': all_test_img_labels,
                  'file_paths': all_test_img_files})
            
    return new_df

In [ ]:
class PNGDataFrameIterator(DataFrameIterator):
    def __init__(self, *arg, **kwargs):
        self.white_list_formats = ('png')
        super(PNGDataFrameIterator, self).__init__(*arg, **kwargs)
        self.dataframe = kwargs['dataframe']
        self.x = self.dataframe[kwargs['x_col']]
        self.y = self.dataframe[kwargs['y_col']]
        self.color_mode = kwargs['color_mode']
        self.target_size = kwargs['target_size']

    def _get_batches_of_transformed_samples(self, indices_array):
        # get batch of images
        batch_x = np.array([self.read_png_as_array(path, self.target_size, 
                                                   color_mode=self.color_mode)
                            for path in self.x.iloc[indices_array]])

        batch_y = np.array(self.y.iloc[indices_array].astype(np.uint8))  # astype because y was passed as str

        # transform images
        if self.image_data_generator is not None:
            for i, (x, y) in enumerate(zip(batch_x, batch_y)):
                transform_params = self.image_data_generator.get_random_transform(x.shape)
                batch_x[i] = self.image_data_generator.apply_transform(x, transform_params)
      

        return batch_x, batch_y

    
    @staticmethod
    def read_png_as_array(path, target_size=(Config.IMG_SIZE, Config.IMG_SIZE),
                          color_mode='rgb'):
        im_gray = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
        pixels = im_gray - np.min(im_gray)
        pixels = pixels / np.max(pixels)
        image_manual_norm = (pixels * 255).astype(np.uint8)
        image_array = cv2.resize(image_manual_norm, target_size, interpolation=cv2.INTER_CUBIC) 
        
        if color_mode == 'rgb':
            image_array = np.dstack((image_array, np.zeros_like(image_array), np.zeros_like(image_array)))
        return image_array

        
    

In [ ]:
exam_list = ['flair','t1w','t1wce','t2w']
# exam_list = ['flair']

dftrainIter_dict = {}
dfvalIter_dict = {}
dftestIter_dict = {}
train_gen_dict = {}
val_gen_dict = {}
test_gen_dict = {}
model_check_dict = {}
model_dict = {}

for exam in exam_list:
    dftrainIter = get_iterating_dataframe(df_train, exam)
    dftrainIter['labels_str'] = dftrainIter['labels'].astype(str)

    dfvalIter = get_iterating_dataframe(df_val, exam)
    dfvalIter['labels_str'] = dfvalIter['labels'].astype(str)

    dftestIter = get_iterating_dataframe(df_test, exam)
    dftestIter['labels_str'] = dftestIter['labels'].astype(str)

    datagen = ImageDataGenerator(
            preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
            zoom_range=0.2,
            rotation_range=45,
            fill_mode='nearest',
            height_shift_range= 0.15,
            width_shift_range=0.15,
            horizontal_flip=True,
            vertical_flip=True,
            brightness_range = [0.8, 1.2],
            rescale=1.0/255,
    )
    test_datagen = ImageDataGenerator(
        preprocessing_function=tf.keras.applications.resnet50.preprocess_input,
        rescale=1.0/255,
    )

    train_generator = PNGDataFrameIterator(dataframe=dftrainIter,
                                     x_col='file_paths',
                                     y_col='labels_str',
                                     image_data_generator=datagen,
                                     seed=Config.SEED,
                                     batch_size=Config.BATCH_SIZE,
                                     class_mode=Config.CLASS_MODE,
                                     color_mode=Config.COLOR_MODE,
                                     target_size=Config.TARGET_SIZE,  
                                    )

    val_generator = PNGDataFrameIterator(dataframe=dfvalIter,
                                     x_col='file_paths',
                                     y_col='labels_str',
                                     image_data_generator=test_datagen,
                                     seed=Config.SEED,
                                     batch_size=Config.BATCH_SIZE,
                                     class_mode=Config.CLASS_MODE,
                                     color_mode=Config.COLOR_MODE,
                                     target_size=Config.TARGET_SIZE,  
                                    )

    test_generator = PNGDataFrameIterator(dataframe=dftestIter,
                                     x_col='file_paths',
                                     y_col='labels_str',
                                     image_data_generator=test_datagen,
                                     seed=Config.SEED,
                                     batch_size=Config.BATCH_SIZE,
                                     class_mode=Config.CLASS_MODE,
                                     color_mode=Config.COLOR_MODE,
                                     target_size=Config.TARGET_SIZE,  
                                    )
    model_resnet = tf.keras.applications.ResNet50(weights='imagenet',
                                              input_shape=(Config.IMG_SIZE, Config.IMG_SIZE, 3),
                                              include_top=False)
    model_effnetb0 = tf.keras.applications.EfficientNetB0(
                                                    weights='imagenet',
                                                  input_shape=(Config.IMG_SIZE, Config.IMG_SIZE, 3),
                                                  include_top=False)

    model_deep = model_resnet
    
    
    # Congela camadas pré-treinadas
#     for layer in model_deep.layers:
#         layer.trainable = False
        
    # Congela todas as camadas exceto as ultimas
    for layer in model_deep.layers[:143]:
#     for layer in model_deep.layers[:165]:
        layer.trainable = False

    # Debug - checa qual layer está congelado    
#     for i, layer in enumerate(model_deep.layers):
#         print( i, layer.name, "-" ,layer.trainable)
        
        
    x = layers.GlobalMaxPooling2D()(model_deep.output)
#     x = layers.Flatten()(model_deep.output)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(32, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    x = layers.BatchNormalization()(x)
#     x = layers.Dense(128, activation='relu')(x)
#     x = layers.Dropout(0.5)(x)
#     x = layers.BatchNormalization()(x)
    out = layers.Dense(1, activation='sigmoid')(x)

    model = Model(inputs=model_deep.input, outputs=out)

    opt = tf.keras.optimizers.Adam(learning_rate=1e-4)

#     model.summary()
    
    model_checkpoint = ModelCheckpoint('../working/resnet2d_congelada_' + exam + '.hdf5', 
                                       monitor='val_loss',verbose=1, 
                                       save_best_only=True)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=10,
                                                      restore_best_weights=True)
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=Config.TENSORBOARD_LOG_DIR,
                                                histogram_freq=1,
                                                )
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5,
                                       patience=2, min_lr=1e-7,
                                      verbose=1)
    #model_ResNet50_2d.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
    model.compile(loss='binary_crossentropy', optimizer=opt, 
                              metrics=[tf.keras.metrics.BinaryAccuracy(),
                                       tf.keras.metrics.AUC(from_logits=False)] )
    

    dftrainIter_dict[exam] = dftrainIter
    dfvalIter_dict[exam] = dfvalIter
    dftestIter_dict[exam] = dftestIter 
    train_gen_dict[exam] = train_generator
    val_gen_dict[exam] = val_generator
    test_gen_dict[exam] = test_generator
    model_check_dict[exam] = model_checkpoint
    model_dict[exam] = model

In [ ]:
model_dict

In [ ]:
# NUM_EPOCHS = 6 * 5 #Should take ~1hr to run for each 5 epochs
# history_dict = {}
# for exam in exam_list:
#     history_dict[exam] = model_dict[exam].fit(train_gen_dict[exam],
#                                     validation_data=val_gen_dict[exam],
#                                     epochs = NUM_EPOCHS,
#                                     verbose = 1,
#                                     callbacks=[
#                                         early_stopping, 
#                                         model_check_dict[exam],
#                                         reduce_lr,
#                                               ]
#                                     )

In [ ]:
# for exam in exam_list:
#     history = history_dict[exam]
#     plt.plot(history.history['loss'])
#     plt.plot(history.history['val_loss'])
#     plt.title('model loss')
#     plt.ylabel('loss')
#     plt.xlabel('epoch')
#     plt.legend(['train', 'val'], loc='upper left')
#     plt.show()

In [ ]:
# #Loading previously trained resnets (5 epochs for now)
!ls ../input/rsna-miccai-resnet50-4exams-5epochs/resnet2d_congelada_' + exam + '.hdf5'
for exam in exam_list:
    model_dict[exam].load_weights('../input/rsna-miccai-resnet50-4exams-5epochs/resnet2d_congelada_' + exam + '.hdf5')

In [ ]:
test_pred_dict = {}
for exam in exam_list:
    print('Computing prediction for ' + exam + '...')
    test_pred = model_dict[exam].predict(test_gen_dict[exam], steps=len(test_gen_dict[exam]))
    dftestIter_dict[exam]['pred_y'] = test_pred
    mean_pred = test_pred.mean()
    test_pred_agg = dftestIter_dict[exam].groupby('patient_ids').apply(
        lambda x: x['pred_y'].max()
        if (x['pred_y'].max() - mean_pred) > (mean_pred - x['pred_y'].min()) 
        else x['pred_y'].min()
    )
    test_pred_dict[exam] = test_pred_agg
print('done!')

In [ ]:
test_pred_dict

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

ns_probs = [0 for _ in range(len(df_test.MGMT_value))]
print(exam_list)
for exam in exam_list:
    ns_probs = [0 for _ in range(len(df_test.MGMT_value))]
    y_label = df_test.MGMT_value
    y_pred  = test_pred_dict[exam]
    ns_auc = roc_auc_score(y_label, ns_probs)
    lr_auc = roc_auc_score(y_label, y_pred)
    print('No skill: ROC AUC=%.3f' % (ns_auc))
    print('CNN: ROC AUC=%.3f' % (lr_auc))
    lr_fpr, lr_tpr, _ = roc_curve(y_label, y_pred)
    plt.plot(lr_fpr, lr_tpr, marker='.', label='CNN')

    ns_fpr, ns_tpr, _ = roc_curve(y_label, ns_probs)
    plt.plot(ns_fpr, ns_tpr, marker='.', label='No Skill')

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()

In [ ]:
#Combining predictions
test_pred_exam_agg = np.zeros(len(df_test.MGMT_value))
for exam in exam_list:
#     if exam is 'flair' or exam is 't2w':
#         pass
#     else:
# #         test_pred_exam_agg = test_pred_exam_agg + test_pred_dict[exam]
#         test_pred_exam_agg = np.maximum( test_pred_exam_agg , test_pred_dict[exam] )
     test_pred_exam_agg = np.maximum( test_pred_exam_agg , test_pred_dict[exam] )
# test_pred_exam_agg = test_pred_exam_agg / 4
test_pred_exam_agg

ns_probs = [0 for _ in range(len(df_test.MGMT_value))]
y_label = df_test.MGMT_value
y_pred  = test_pred_exam_agg
ns_auc = roc_auc_score(y_label, ns_probs)
lr_auc = roc_auc_score(y_label, y_pred)
print('No skill: ROC AUC=%.3f' % (ns_auc))
print('CNN: ROC AUC=%.3f' % (lr_auc))
lr_fpr, lr_tpr, _ = roc_curve(y_label, y_pred)
plt.plot(lr_fpr, lr_tpr, marker='.', label='CNN')

ns_fpr, ns_tpr, _ = roc_curve(y_label, ns_probs)
plt.plot(ns_fpr, ns_tpr, marker='.', label='No Skill')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
val_pred_dict = {}
for exam in exam_list:
    print('Computing prediction for ' + exam + '...')
    val_pred = model_dict[exam].predict(val_gen_dict[exam], steps=len(val_gen_dict[exam]))
    dfvalIter_dict[exam]['pred_y'] = val_pred
    mean_pred = val_pred.mean()
    val_pred_agg = dfvalIter_dict[exam].groupby('patient_ids').apply(
        lambda x: x['pred_y'].max()
        if (x['pred_y'].max() - mean_pred) > (mean_pred - x['pred_y'].min()) 
        else x['pred_y'].min()
    )
    val_pred_dict[exam] = val_pred_agg
print('done!')

val_pred_dict

from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

ns_probs = [0 for _ in range(len(df_val.MGMT_value))]
print(exam_list)
for exam in exam_list:
    ns_probs = [0 for _ in range(len(df_val.MGMT_value))]
    y_label = df_val.MGMT_value
    y_pred  = val_pred_dict[exam]
    ns_auc = roc_auc_score(y_label, ns_probs)
    lr_auc = roc_auc_score(y_label, y_pred)
    print('No skill: ROC AUC=%.3f' % (ns_auc))
    print('CNN: ROC AUC=%.3f' % (lr_auc))
    lr_fpr, lr_tpr, _ = roc_curve(y_label, y_pred)
    plt.plot(lr_fpr, lr_tpr, marker='.', label='CNN')

    ns_fpr, ns_tpr, _ = roc_curve(y_label, ns_probs)
    plt.plot(ns_fpr, ns_tpr, marker='.', label='No Skill')

    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.show()

#Combining predictions
val_pred_exam_agg = np.zeros(len(df_val.MGMT_value))
for exam in exam_list:
#     if exam is 'flair' or exam is 't2w':
#         pass
#     else:
# #         val_pred_exam_agg = val_pred_exam_agg + val_pred_dict[exam]
#         val_pred_exam_agg = np.maximum( val_pred_exam_agg , val_pred_dict[exam] )
     val_pred_exam_agg = np.maximum( val_pred_exam_agg , val_pred_dict[exam] )
# val_pred_exam_agg = val_pred_exam_agg / 4
val_pred_exam_agg

ns_probs = [0 for _ in range(len(df_val.MGMT_value))]
y_label = df_val.MGMT_value
y_pred  = val_pred_exam_agg
ns_auc = roc_auc_score(y_label, ns_probs)
lr_auc = roc_auc_score(y_label, y_pred)
print('No skill: ROC AUC=%.3f' % (ns_auc))
print('CNN: ROC AUC=%.3f' % (lr_auc))
lr_fpr, lr_tpr, _ = roc_curve(y_label, y_pred)
plt.plot(lr_fpr, lr_tpr, marker='.', label='CNN')

ns_fpr, ns_tpr, _ = roc_curve(y_label, ns_probs)
plt.plot(ns_fpr, ns_tpr, marker='.', label='No Skill')

plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()


def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.5):
    # Load the original image
#     img = keras.preprocessing.image.load_img(img_path)
#     img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
#     display(Image(cam_path))




def gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=None,
           last_conv_layer_name='conv5_block3_3_conv',
           output_prefix = None):

    # Prepare image
    if model is None:
        model_builder = keras.applications.ResNet50
        model = model_builder(weights="imagenet")
    
    
    img_size = (Config.IMG_SIZE, Config.IMG_SIZE)

    for i, img_path in enumerate(files_paths):
        # The local path to our target image
        img_array = preprocess_input(get_img_array(img_path, size=img_size))

        model.layers[-1].activation = None

        # Generate class activation heatmap
        heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name)
        

        output_path = '../working/gradCam_out_' + output_prefix + '_' + str(i) +'.png'
        
        save_and_display_gradcam(img_array[0], heatmap, output_path)

In [ ]:
files_paths = [
    '../input/rsna-miccai-png/train/00000/FLAIR/Image-242.png',
    '../input/rsna-miccai-png/train/00045/FLAIR/Image-116.png',
     '../input/rsna-miccai-png/train/00133/FLAIR/Image-35.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['flair'],
            output_prefix = 'flair'
           )

In [ ]:
files_paths = [
    '../input/rsna-miccai-png/train/00000/T2w/Image-126.png',
    '../input/rsna-miccai-png/train/00045/T2w/Image-323.png',
     '../input/rsna-miccai-png/train/00133/T2w//Image-19.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t2w'],
            output_prefix = 't2w'
           )

In [ ]:
files_paths = [
    '../input/rsna-miccai-png/train/00000/T1wCE/Image-78.png',
    '../input/rsna-miccai-png/train/00045/T1wCE/Image-77.png',
     '../input/rsna-miccai-png/train/00133/T1wCE//Image-35.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t1wce'],
            output_prefix = 't1wce'
           )

In [ ]:
files_paths = [
    '../input/rsna-miccai-png/train/00478/FLAIR/Image-100.png',
    '../input/rsna-miccai-png/train/00478/FLAIR/Image-105.png',
     '../input/rsna-miccai-png/train/00478/FLAIR/Image-113.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['flair'],
            output_prefix = 'flair_00478'
           )

In [ ]:
files_paths = [
    '../input/rsna-miccai-png/train/00197/T1wCE/Image-105.png',
    '../input/rsna-miccai-png/train/00197/T1wCE/Image-110.png',
     '../input/rsna-miccai-png/train/00197/T1wCE/Image-118.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t1wce'],
            output_prefix = 't1wce_00197'
           )

#!rm -f 
files_paths = [
    '../input/rsna-miccai-png/train/00011/T1wCE/Image-48.png',
    '../input/rsna-miccai-png/train/00011/T1wCE/Image-55.png',
     '../input/rsna-miccai-png/train/00011/T1wCE/Image-66.png',
     '../input/rsna-miccai-png/train/00011/T1wCE/Image-86.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t1wce'],
            output_prefix = 't1wce_00011'
           )

files_paths = [
    '../input/rsna-miccai-png/train/00500/T1wCE/Image-196.png',
    '../input/rsna-miccai-png/train/00500/T1wCE/Image-216.png',
     '../input/rsna-miccai-png/train/00500/T1wCE/Image-232.png',
     '../input/rsna-miccai-png/train/00500/T1wCE/Image-252.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t1wce'],
            output_prefix = 't1wce_00500'
           )

files_paths = [
    '../input/rsna-miccai-png/train/00011/T1wCE/Image-48.png',
    '../input/rsna-miccai-png/train/00011/T1wCE/Image-55.png',
     '../input/rsna-miccai-png/train/00011/T1wCE/Image-66.png',
     '../input/rsna-miccai-png/train/00011/T1wCE/Image-86.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['flair'],
            output_prefix = 's_t1wce_m_flair_00011'
           )


In [ ]:
#ruim
files_paths = [
    '../input/rsna-miccai-png/train/00500/FLAIR/Image-84.png',
    '../input/rsna-miccai-png/train/00500/FLAIR/Image-104.png',
     '../input/rsna-miccai-png/train/00500/FLAIR/Image-146.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['flair'],
            output_prefix = 'flair_00500'
           )

#bom
files_paths = [
    '../input/rsna-miccai-png/train/00201/FLAIR/Image-22.png',
    '../input/rsna-miccai-png/train/00201/FLAIR/Image-26.png',
     '../input/rsna-miccai-png/train/00201/FLAIR/Image-36.png',
     '../input/rsna-miccai-png/train/00201/FLAIR/Image-42.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['flair'],
            output_prefix = 'flair_00201'
           )

files_paths = [
    '../input/rsna-miccai-png/train/00201/FLAIR/Image-22.png',
    '../input/rsna-miccai-png/train/00201/FLAIR/Image-26.png',
     '../input/rsna-miccai-png/train/00201/FLAIR/Image-36.png',
     '../input/rsna-miccai-png/train/00201/FLAIR/Image-42.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t1wce'],
            output_prefix = 's_flair_m_t1wce_00201'
           )



In [ ]:
# !ls
# !rm *.png

In [ ]:
#bem ruim
files_paths = [
    '../input/rsna-miccai-png/train/00066/T1w/Image-11.png',
    '../input/rsna-miccai-png/train/00066/T1w/Image-18.png',
     '../input/rsna-miccai-png/train/00066/T1w/Image-20.png',
     '../input/rsna-miccai-png/train/00066/T1w/Image-24.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t1w'],
            output_prefix = 't1w_00066'
           )

files_paths = [
    '../input/rsna-miccai-png/train/00414/T1w/Image-102.png',
    '../input/rsna-miccai-png/train/00414/T1w/Image-115.png',
     '../input/rsna-miccai-png/train/00414/T1w/Image-94.png',
     '../input/rsna-miccai-png/train/00414/T1w/Image-89.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t1w'],
            output_prefix = 't1w_00414'
           )


In [ ]:
files_paths = [
    '../input/rsna-miccai-png/train/00094/T2w/Image-128.png',
    '../input/rsna-miccai-png/train/00094/T2w/Image-224.png',
     '../input/rsna-miccai-png/train/00094/T2w/Image-281.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t2w'],
            output_prefix = 't2w_00094'
           )

files_paths = [
    '../input/rsna-miccai-png/train/00191/T2w/Image-48.png',
    '../input/rsna-miccai-png/train/00191/T2w/Image-38.png',
     '../input/rsna-miccai-png/train/00191/T2w/Image-32.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t2w'],
            output_prefix = 't2w_00191'
           )

files_paths = [
    '../input/rsna-miccai-png/train/00298/T2w/Image-31.png',
    '../input/rsna-miccai-png/train/00298/T2w/Image-35.png',
     '../input/rsna-miccai-png/train/00298/T2w/Image-40.png',
     '../input/rsna-miccai-png/train/00298/T2w/Image-47.png',
    
]
gradCAM(files_paths, 
            preprocess_input=keras.applications.resnet50.preprocess_input,
            model=model_dict['t2w'],
            output_prefix = 't2w_00298'
           )


In [ ]:
test_pred_dict['t1w'].head(50)

In [ ]:
!zip -r gradcam_pngs.zip *png

In [ ]:
from IPython.display import FileLink
FileLink(r'gradcam_pngs.zip')